# Machine Learning On Tennis Data
## Contents
>* [Setup](#0)
>* [Model Fitting](#1)

## 0 - Setup <a class="anchor" id="0"></a>

In [1]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb

In [2]:
f = open('train_test_split.pkl', 'rb')
X_train, X_test, y_train, y_test = pickle.load(f)

In [3]:
X_train

,A_rank,B_rank,A_hist_form,A_form,A_court_form,A_surface_form,B_form,B_court_form,B_surface_form,ranking_p,...,B_Zverev A.,B_Zverev M.,B_de Chaunac S.,B_de Voest R.,B_di Mauro A.,B_di Pasquale A.,B_van Gemerden M.,B_van Lottum J.,B_van Scheppingen D.,B_van der Meer N.
39960,9.0,11.0,0.666667,0.466667,1.0,0.0,0.200000,1.0,0.0,0.100000,...,0,0,0,0,0,0,0,0,0,0
5803,6.0,93.0,1.000000,0.600000,0.0,1.0,0.066667,0.0,1.0,0.878788,...,0,0,0,0,0,0,0,0,0,0
35357,91.0,52.0,0.500000,-0.200000,1.0,0.0,-0.466667,1.0,0.0,-0.272727,...,0,0,0,0,0,0,0,0,0,0
12532,123.0,103.0,0.500000,-0.200000,1.0,1.0,-0.066667,1.0,0.0,-0.088496,...,0,0,0,0,0,0,0,0,0,0
51066,52.0,40.0,0.500000,-0.066667,1.0,0.0,-0.200000,1.0,0.0,-0.130435,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11323,16.0,58.0,0.500000,0.066667,0.0,0.0,0.066667,0.0,0.0,0.567568,...,0,0,0,0,0,0,0,0,0,0
44948,58.0,84.0,0.500000,0.200000,0.0,1.0,-0.200000,1.0,1.0,0.183099,...,0,0,0,0,0,0,0,0,0,0
38336,73.0,131.0,0.500000,-0.200000,1.0,1.0,-0.466667,1.0,1.0,0.284314,...,0,0,0,0,0,0,0,0,0,0
863,448.0,51.0,0.500000,0.500000,0.0,0.0,-0.333333,1.0,1.0,-0.795591,...,0,0,0,0,0,1,0,0,0,0


In [4]:
y_train

39960    0
5803     1
35357    1
12532    1
51066    1
        ..
11323    0
44948    0
38336    1
863      0
15850    1
Name: y, Length: 39902, dtype: int64

## 1 - Model Fitting <a class="anchor" id="1"></a>

In [8]:
## XGBOOST
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [9]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

evallist = [(dtest, 'eval'), (dtrain, 'train')]

In [10]:
num_round = 10
bst = xgb.train(param, dtrain, num_round, evallist)

[0]	eval-auc:0.69550	train-auc:0.69699
[1]	eval-auc:0.69943	train-auc:0.70142
[2]	eval-auc:0.70859	train-auc:0.71098
[3]	eval-auc:0.70995	train-auc:0.71478
[4]	eval-auc:0.71037	train-auc:0.71628
[5]	eval-auc:0.71069	train-auc:0.71701
[6]	eval-auc:0.71385	train-auc:0.71923
[7]	eval-auc:0.71475	train-auc:0.71984
[8]	eval-auc:0.71695	train-auc:0.72147
[9]	eval-auc:0.71723	train-auc:0.72221
